In [11]:
import numpy as np
import pandas as pd
import random

In [12]:
def generate_rand_init_pop():
    
    """ 
    This function generates n random initial candidate solutions, constituting the initial population

    Parameters: 
        None

    Returns: 
        pop (DataFrame): DataFrame containing all n random initial candidate solutions
    """
    
    # Prompt user to indicate the number of initial candidate solutions required 
    print('Number of initial candidate solutions required? (Enter int):')
    
    n = int(input())
    
    # Declare population DataFrame
    pop = pd.DataFrame(columns=['solution'])
    
    # Generate n random candidate solutions
    for i in range(n):
        
        #Generate a random solution of length 1x6 selecting integers 0 through 5 without duplication
        sol = np.array(random.sample(range(6), 6))
        
        # Append population DataFrame with random solution
        pop = pop.append({'solution': sol}, ignore_index=True)
        
    return pop

In [13]:
def fitness(pop, dist_mat):
    
    """ 
    This function calculates the fitness value and associated probability of selection of a provided solution(path) of the TSP

    Parameters: 
        pop (DataFrame): Dataframe consisting of population of all candidate solutions
        dist_mat (array): Distance matrix from which distances are obtained

    Returns: 
        pop_f (DataFrame): DataFrame of type pop but including fitness value of each candidate solution (distance travelled) 
        and it's proportional probability of selection
    """
    
    # Make a copy of pop to append to 
    pop_f = pop.copy()
    
    # Declare variable for fitness value
    f = 0
    
    # Iterate over all candidate solutions in population
    for j in range(len(pop)):
        
        # Reset fitness value, f
        f = int(0)
        
        # Assign current candidate solution to sol
        sol = pop['solution'][j]

        # Iterate over all cities (points) in the candidate solution
        for i in range(len(sol)):

            # Determine the cities (points) between which distances must be added to the fitness value
            city1 = sol[i-1]
            city2 = sol[i]

            # Increase the fitness value by distance obtained from the distance matrix 
            f = f + dist_mat[city1,city2]
            
        # Append pop DataFrame with current candidate solution fitness value
        pop_f.loc[j,'f'] = int(f)
        
        # Calculate probabilities for selection by taking the inverse of distances travelled 
        # and converting the inverses to ensure a cumulative probability of 1
        pop_f['f_inv'] = 1 / pop_f['f']
        pop_f['prob'] = pop_f['f_inv'] / pop_f['f_inv'].sum()
     
    return pop_f

In [14]:
def roulette(pop_f):
    
    """ 
    This function employs roulette selection of given candidate solutions according to the process outlined by Pencheva, Atanassov & Shannon (2020)

    Parameters: 
        pop_f (DataFrame): DataFrame including fitness value of each candidate solution (distance travelled) and its proportional probability of selection

    Returns: 
        roulette_winner (int): DataFrame index of winner of the roulette selection
    """
    
    # Generate random number between 0 and 1 sampling a uniform distribution (winning probability)
    rand_prob = random.uniform(0, 1)

    # Declare variable for cumulating probabilities
    cum_prob = 0
    
    # Recalculate probabilities:
    pop_f['f_inv'] = 1 / pop_f['f']
    pop_f['prob'] = pop_f['f_inv'] / pop_f['f_inv'].sum()

    # Iterate over all candidate solutions to calculate and append cumulative probabilities, i.e. mapped to contiguous segments of a line
    for i in range(len(pop_f)):

        # Calculate and add cumulative probability to each candidate solution
        cum_prob = cum_prob + pop_f.loc[i,'prob']
        pop_f.loc[i,'cum_prob'] = cum_prob

    # Iterate over all candidate solutions
    for i in range(len(pop_f)):

        # If randomly generated probability is less than the first segment, index 0 wins and for loop is terminated
        if rand_prob < pop_f.loc[i,'cum_prob']:
            roulette_winner = pop_f.loc[i,'index']
            break

        # If randomly generated probability is more than the previous segment, index i + 1 wins
        elif rand_prob >= pop_f.loc[i,'cum_prob']:

            # If randomly generated probability is 1, make winner = max index
            if rand_prob == 1: 
                roulette_winner = len(pop_f) - 1

            # Winner is the index that has a larger corresponding cumulative probability
            roulette_winner = pop_f.loc[i,'index']
            
    return roulette_winner

In [15]:
def tournament(pop_f, k):

    """ 
    This function selects 6 solutions by performing tournament selection on 8 candidate solutions with removal

    Parameters: 
        pop_f (DataFrame): DataFrame including population of candidate solutions and fitness values (distance travelled) of each
        k (int): Tournament size

    Returns: 
        winners (list): List of indices of winning 6 candidate solutions
        losers (list): List of indices of losing 2 candidate solutions for use in next generation 
    """

    # Make a copy of pop_f to alter during candidate removal and for use in tournament
    # making a copy of the initial index using rest.index() to keep track of original index
    pop_f_tourn = pop_f.copy().reset_index()

    # Declare list to store winning and losing candidate solutions
    winners = []
    losers = []

    # Perform 6 iterations to select 6 solutions 
    for i in range(6):
        
        # Declare list to store players selected through roulette selection
        players = []
        
        # Make a copy of pop_f_tourn to be used during roulette selection 
        pop_f_roul = pop_f_tourn.copy().reset_index(drop=True)
        
        # Utilise roulette selection k times to select k players
        for j in range(k):
            
            player = roulette(pop_f_roul)

            players.append(player)
            
            # Create DataFrame of all 6 players
            pop_f_roul = pop_f_roul[pop_f_roul['index'] != player].reset_index(drop=True)

        # Select winner based on lowest distance to be travelled and assign index to variable winner
        player_df = pop_f_tourn[pop_f_tourn['index'].isin(players)]
        winner = player_df.sort_values(by=['f']).reset_index(drop=True)['index'].loc[0]

        # Append winning candidate solution and fitness value to winners DataFrame
        winners.append(winner)

        # Remove winner of this tournament from available candidate solutions for the next tournament 
        # and reset DataFrame index
        pop_f_tourn = pop_f_tourn[pop_f_tourn['index'] != winner]
        
    # Store the remaining 2 solutions in a list 
    losers = list(pop_f_tourn['index'])

    return winners, losers

In [16]:
def crossover(parent1, parent2):
    
    """ 
    This function performs 2-point crossover to a pair of parent solutions producing 2 offspring solutions

    Parameters: 
        parent1 (array): 1xn array of the first parent
        parent2 (array): 1xn array of the second parent

    Returns: 
        offspring1 (array): 1xn array of the first offspring generated
        offspring2 (array): 1xn array of the second offspring generated
    """    
    # Declare offspring array
    offspring1 = np.zeros((6,), dtype=int)
    offspring2 = np.zeros((6,), dtype=int)

    # Generate 2 random points for crossover. Assign min to crossover_point1 and max to crossover_point2
    crossover_points = np.sort(np.array(random.sample(range(0, 7), 2)))
    crossover_point1 = crossover_points[0]
    crossover_point2 = crossover_points[1]

    # CREATION OF OFFSPRING 1
    # Determine entries that are retained from parent 1
    crossover1 = np.concatenate([parent1[:crossover_point1],parent1[crossover_point2:]])
    
    # Determine unique entries, in order, that are not part of crossover1 and in parent2
    not_in_crossover1 = np.setdiff1d(parent2, crossover1, assume_unique=True)
    
    # Copy parent1 and replace relevant entries from those in parent 2
    offspring1 = np.copy(parent1)
    for i in range(len(not_in_crossover1)):
        
        offspring1[crossover_point1+i] = not_in_crossover1[i]
        
    # CREATION OF OFFSPRING 2
    # Determine entries that are retained from parent 2
    crossover2 = np.concatenate([parent2[:crossover_point1],parent2[crossover_point2:]])
    
    # Determine unique entries, in order, that are not part of crossover2 and in parent1
    not_in_crossover2 = np.setdiff1d(parent1, crossover2, assume_unique=True)
    
    # Copy parent2 and replace relevant entries from those in parent 1
    offspring2 = np.copy(parent2)
    for i in range(len(not_in_crossover2)):
        
        offspring2[crossover_point1+i] = not_in_crossover2[i]
    
    return offspring1, offspring2

In [17]:
def mutation(sol):
    
    """ 
    This function mutates a candidate solution by swapping 2 random entries (cities)

    Parameters: 
        sol (array): 1xn array of solution to be mutated
        
    Returns: 
        sol (array): 1xn array of mutated solution
    """    
    
    # Randomly select 2 entries (cities)
    swap_entries = np.array(random.sample(range(6), 2))
    entry0 = sol[swap_entries[0]]
    entry1 = sol[swap_entries[1]]
    
    # Swap entries of candidate solution
    sol[swap_entries[0]] = entry1
    sol[swap_entries[1]] = entry0
    
    return sol

In [18]:
def offspring_generation(pop_f, winners):
    
    """ 
    This function generates all offspring making use of the crossover(parent1, parent2) function
    This is followed by mutation of one randomly selected offspring solution

    Parameters: 
        pop_f (DataFrame): DataFrame including population of candidate solutions and fitness values (distance travelled) of each
        winners (list): List of parent solutions to be used for offspring solution generation

    Returns: 
        new_pop (DataFrame): DataFrame including all parent and offspring candidate solutions 
    """    
    
    # Declare DataFrame with all parent and offspring solutions
    new_pop = pd.DataFrame(columns=['solution'],index=list(range(12)))
    
    # Retain only winners and then shuffle solutions to ensure random pairing
    pop_f_winners = pop_f.loc[winners]
    pop_f_shuffled = pop_f_winners.sample(frac=1).reset_index(drop=True)

    
    # Separate current candidate solution arrays and allocate them to parent variables
    parent_0 = pop_f_shuffled.loc[0,'solution']
    parent_1 = pop_f_shuffled.loc[1,'solution']
    parent_2 = pop_f_shuffled.loc[2,'solution']
    parent_3 = pop_f_shuffled.loc[3,'solution']
    parent_4 = pop_f_shuffled.loc[4,'solution']
    parent_5 = pop_f_shuffled.loc[5,'solution']
    
    # Generate offspring using 2-point crossover calling the crossover function
    offspring_0, offspring_1 = crossover(parent_0, parent_1)
    offspring_2, offspring_3 = crossover(parent_2, parent_3)
    offspring_4, offspring_5 = crossover(parent_4, parent_5) 

    
    # Iterate over parents and offspring (all candidate solutions) and assign solutions to DataFrame to be returned
    for i in range(len(pop_f_winners)):
        
        # Create strings for variables so that their values can be addressed during the loop
        parent_str = 'parent_' + str(i)
        offspring_str = 'offspring_' + str(i)
        
        # Update DataFrame with the all offspring solutions and variable name
        new_pop.loc[i,'solution'] = eval(offspring_str)
        new_pop.loc[i,'Par/Off/Prev'] = offspring_str       
            
        # Update DataFrame with the all parent solutions, indicating which were used for crossover
        new_pop.loc[i+6,'solution'] = eval(parent_str)
        if i <= 1: new_pop.loc[i+6,'Par/Off/Prev'] = 'parent_pair_1'
        elif i <= 3: new_pop.loc[i+6,'Par/Off/Prev'] = 'Parent_pair_2'
        elif i <= 5: new_pop.loc[i+6,'Par/Off/Prev'] = 'Parent_pair_3'

    # Mutation of randomly selected offspring and overwrite the previous solution
    offspring_mut = int(np.array(random.sample(range(6), 1)))
    new_pop.loc[offspring_mut,'solution'] = mutation(new_pop.loc[offspring_mut,'solution'])
    
    # Record which offspring was mutated
    new_pop.loc[offspring_mut,'Mutated?'] = 'yes'
    
    return new_pop.reset_index(drop=True)

In [30]:
def main():
    
    """ 
    This function runs executes all functions until the stopping condition in met
    Stopping condition: no improvement for 10 consecutive iterations

    Parameters: None

    Returns: 
        pop_f (DataFrame): DataFrame including all candidate solutions and supporting information
    """
    
    # Intitialise the provided distance matrix
    dist_mat = np.array([[0, 41, 26, 31, 27, 35],
                         [41, 0, 29, 32, 40, 33],
                         [26, 29, 0, 25, 34, 42],
                         [31, 32, 25, 0, 28, 34],
                         [27, 40, 34, 28, 0, 36],
                         [35, 33, 42, 34, 36, 0]])
    
    print('-----------------------------------------------------')
    
    # Generate random population
    pop = generate_rand_init_pop()
    
    # Calculate fitness values and probabilities for selection in roulette selection
    pop_f = fitness(pop, dist_mat)
    
    # Declare generation counter, stopping criteria counter and stop flag
    g = 0
    stop_counter = 0
    stop_flag = False
    
    print('-----------------------------------------------------')
    
    while stop_flag == False:
        
        # Print generation number
        print("Generation", g)

        # Record current incumbent and fitness value
        incumbent_sol = pop_f.sort_values(by=['f']).reset_index(drop=True)['solution'].loc[0]
        incumbent_f = pop_f.sort_values(by=['f']).reset_index(drop=True)['f'].loc[0]

        # Select winners and losers by tournament selection
        winners, losers = tournament(pop_f, 3)

        # Generate offspring using 2-point crossover and randomly mutate one offspring (new_pop includes all solutions)
        new_pop = offspring_generation(pop_f, winners)
        
        # Append 2 losing solutions to new population and state that it is a previous solution
        new_pop.loc[12,'solution'] = pop_f.loc[losers[0], 'solution']
        new_pop.loc[13,'solution'] = pop_f.loc[losers[1], 'solution']
        new_pop.loc[12,'Par/Off/Prev'] = 'Previous'
        new_pop.loc[13,'Par/Off/Prev'] = 'Previous'

        # Calculate fitness values and probabilities for all solutions
        new_pop_f = fitness(new_pop, dist_mat)
        
        # Print all solutions and fitness value mean
        print(new_pop_f.fillna('no').drop(['f_inv', 'prob'], axis=1))
        print("Mean Population Fitness = ", round(new_pop_f['f'].mean(),2))
        print('-----------------------------------------------------')
        
        # Remove duplicate solutions for the next round
        list = []
        for i in range(len(new_pop_f)):
            list.append(np.array2string(new_pop_f.loc[i,'solution']))
        new_pop_f['sol'] = list
        new_pop_f = new_pop_f.drop_duplicates(subset='sol')
        new_pop_f = new_pop_f.drop(['sol'], axis=1)
            
        # Use elitism to select top 8 candidate solutions for the next generation
        new_pop_f = new_pop_f.sort_values(by=['f'])[0:8].reset_index(drop=True)
        
        # Recalculate fitness values and probabilities for the next generation to ensure correct cumulative probabilites
        pop_f = fitness(new_pop_f.drop(['f', 'f_inv', 'prob'], axis = 1), dist_mat)

        # Check stopping condition
        if pop_f.loc[0]['f'] >= incumbent_f:
            stop_counter = stop_counter + 1
        else:
            stop_counter = 0
            
        if stop_counter == 10:
            stop_flag = True
        
        # Increment generation counter
        g = g + 1
        
    print("Optimal path to follow: ", incumbent_sol)
    print("Shortest distance = ",incumbent_f)         
    
    return pop_f

In [39]:
pop_f = main()

-----------------------------------------------------
Number of initial candidate solutions required? (Enter int):


 8


-----------------------------------------------------
Generation 0
              solution   Par/Off/Prev Mutated?      f
0   [4, 2, 0, 1, 5, 3]    offspring_0       no  196.0
1   [4, 5, 0, 2, 1, 3]    offspring_1       no  186.0
2   [0, 5, 4, 3, 2, 1]    offspring_2       no  194.0
3   [3, 0, 5, 1, 2, 4]    offspring_3      yes  190.0
4   [1, 2, 4, 0, 5, 3]    offspring_4       no  191.0
5   [2, 1, 4, 3, 0, 5]    offspring_5       no  205.0
6   [4, 2, 0, 1, 5, 3]  parent_pair_1       no  196.0
7   [4, 5, 0, 2, 1, 3]  parent_pair_1       no  186.0
8   [0, 5, 4, 3, 2, 1]  Parent_pair_2       no  194.0
9   [3, 0, 2, 1, 5, 4]  Parent_pair_2       no  183.0
10  [1, 2, 0, 4, 5, 3]  Parent_pair_3       no  184.0
11  [2, 1, 3, 4, 0, 5]  Parent_pair_3       no  193.0
12  [1, 4, 3, 2, 5, 0]       Previous       no  211.0
13  [2, 0, 1, 4, 3, 5]       Previous       no  211.0
Mean Population Fitness =  194.29
-----------------------------------------------------
Generation 1
              solution